<a href="https://colab.research.google.com/github/IvenoCarolus/kaggle_global_warming_sentiment_predict/blob/master/Just_a_lil_something_something.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Second attempt, but with a different text cleaning function.**

In [36]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Geting the twitter data (train,test and sample csv files)**

In [0]:
sample_sub_df = pd.read_csv('https://raw.githubusercontent.com/IvenoCarolus/kaggle_global_warming_sentiment_predict/master/data/sample_submission.csv',error_bad_lines=False)
train_df = pd.read_csv('https://raw.githubusercontent.com/IvenoCarolus/kaggle_global_warming_sentiment_predict/master/data/train.csv',error_bad_lines=False)
test_df = pd.read_csv('https://raw.githubusercontent.com/IvenoCarolus/kaggle_global_warming_sentiment_predict/master/data/test.csv',error_bad_lines=False)

In [16]:
train_df.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [18]:
test_df.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


## **Cleaning up the tweet data from the message column**

In [0]:
corpus = []
for tweet in train_df['message'].values:
  #Text cleaning
  cleaned_tweet = re.sub(r'\W', ' ', str(tweet)) #removing all special characters
  cleaned_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', cleaned_tweet) #removing single characters
  cleaned_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', cleaned_tweet)  #removing unnecessary spaces
  cleaned_tweet = re.sub(r'^b\s+', '', cleaned_tweet)           #removing substring prefixed b
  cleaned_tweet = cleaned_tweet.lower()
  cleaned_tweet = cleaned_tweet.split()
  all_stopwords = stopwords.words('english')
  #cleaned_tweet = [stemmer.lemmatize(word) for word in cleaned_tweet] #does the same thing as PorterStemmer kinda,thus the performance doesnt really change it seems
  ps = PorterStemmer()
  cleaned_tweet = [ps.stem(word) for word in cleaned_tweet if not word in set(all_stopwords)] #stemming and making sure all stopwords from the english set is not included in our set
  #cleaned_tweet = [stemmer.lemmatize(word) for word in cleaned_tweet if not word in set(all_stopwords)]
  cleaned_tweet = ' '.join(cleaned_tweet)
  corpus.append(cleaned_tweet)

## **Text-to-Sequence process**

In [0]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray()

## **Train Test splitting**

In [0]:
y = np.array(train_df['sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## **Fitting model with train data**

In [35]:
classifier = LogisticRegression(C=20)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=20, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## **Classification report**

In [41]:
print(classification_report(y_test, classifier.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.73      0.45      0.56       249
           0       0.57      0.41      0.48       473
           1       0.75      0.86      0.80      1701
           2       0.76      0.76      0.76       741

    accuracy                           0.73      3164
   macro avg       0.70      0.62      0.65      3164
weighted avg       0.73      0.73      0.72      3164

